# CloudWatchQuery

> Building and executing queries to CloudWatch Logs

## Query CloudWatchLogs 

CloudWatch Logs support a [simple query language](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/CWL_QuerySyntax.html), which we can use to search for different pattern in the logs.

We will start with some examples and then create the classes and functions to perform these queries.

In [ ]:
simple_query = f'''
fields @timestamp, @message
| sort @timestamp desc
| limit 20 
'''

In [ ]:
complex_query = f'''
filter @message like /GOING TO RUN SKILL/ 
| parse @message '"*" for SESSION *:* with PARAMETERS' as @skill, @channel, @id 
| stats count(@channel) by @skill
'''

In [ ]:
from datetime import datetime, timedelta
import time


In [ ]:
%pip install --upgrade boto3 awswrangler --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import awswrangler as wr
import pandas as pd
from datetime import datetime

In [ ]:
LOG_GROUP_NAME = '/aws/apigateway/welcome'

In [ ]:
import boto3
boto3.setup_default_session(region_name="us-east-2")

In [ ]:
df = wr.cloudwatch.read_logs(
    log_group_names=[LOG_GROUP_NAME],
    query="fields @timestamp, @message | sort @timestamp desc | limit 5",
    boto3_session = boto3.Session(region_name="us-west-2")
)

In [ ]:
df

,timestamp,message,ptr
0,2021-12-20 11:03:13.568,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
1,2021-11-24 15:10:19.572,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
2,2021-11-01 15:57:57.790,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
3,2021-11-01 15:37:36.574,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
4,2021-10-30 23:10:28.807,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...


In [ ]:
datetime.today()- timedelta(days=14)

datetime.datetime(2022, 9, 18, 17, 11, 43, 891593)

In [ ]:
#| default_exp CloudWatchQuery

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import awswrangler as wr
import pandas as pd
from datetime import datetime, timedelta
import boto3

class CloudWatchQuery:
    "Configure a  CloudWatch query"
    def __init__(self, query:str, log_group:str): 
        self.query = query 
        self.log_groups = []
        self.log_groups.append(log_group)
        self.region = "us-west-2"
        
    def add_log_group(self, log_group:str):
        self.log_groups.append(log_group)
        
    def set_region(self, region:str):
        self.region = region

    def get_df(self, 
              days_start:int = 14, # number of days in the past to start
              hours_start:int = 0, # number of hours in the past to start
              days_end:int = 0, # number of days in the past to end
              hours_end:int = 0): # number of hours in the past to end
        "Execute the query"
        df = wr.cloudwatch.read_logs(
            log_group_names=self.log_groups,
            query=self.query,
            start_time=datetime.now()- timedelta(days=days_start, hours=hours_start),
            end_time=datetime.now()- timedelta(days=days_end, hours=hours_end),

            boto3_session = boto3.Session(region_name=self.region)
        )
        return df

In [ ]:
#| export
from fastcore.foundation import patch

@patch
def set_region(self:CloudWatchQuery, region:str):
    "Set the region of the query."
    self.region = region

In [ ]:
cwq = CloudWatchQuery(simple_query, LOG_GROUP_NAME)

In [ ]:
cwq.get_df()

""


In [ ]:
cwq.get_df(days_start=365, hours_start=7)

,timestamp,message,ptr
0,2021-12-20 11:03:13.568,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
1,2021-11-24 15:10:19.572,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
2,2021-11-01 15:57:57.790,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
3,2021-11-01 15:37:36.574,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
4,2021-10-30 23:10:28.807,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
5,2021-10-25 17:55:32.019,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...
6,2021-10-25 15:46:26.526,Cloudwatch logs enabled for API Gateway,CmAKKAokMzI3MTYzODM3MDU1Oi9hd3MvYXBpZ2F0ZXdheS...


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()